In [1]:
import pandas as pd
import plotly.express as px

In [2]:
beds = pd.read_csv('data-ingest/data/beds_hb_wide_unfiltered.csv', parse_dates = [0])
beds.head()

,Date,health_board,CO_GABed_C19,CO_GABed_NonC19,CO_GABed_Spare,CO_GABed_Tot,CO_IVBed_C19,CO_IVBed_NonC19,CO_IVBed_Spare,CO_IVBed_Tot
0,2020-04-01,Betsi Cadwaladr University Health Board,93,656,499,1248,14,5,25,44
1,2020-04-01,Hywel Dda University Health Board,68,312,445,825,5,9,32,46
2,2020-04-01,Abertawe Bro Morgannwg University Health Board,100,515,423,1038,23,10,58,91
3,2020-04-01,Cardiff and Vale University Health Board,158,510,683,1351,19,13,31,63
4,2020-04-01,Cwm Taf University Health Board,111,486,353,950,16,7,34,57


In [3]:
beds.health_board.unique()

array(['Betsi Cadwaladr University Health Board',
       'Hywel Dda University Health Board',
       'Abertawe Bro Morgannwg University Health Board',
       'Cardiff and Vale University Health Board',
       'Cwm Taf University Health Board',
       'Aneurin Bevan University Health Board',
       'Velindre University NHS Trust', 'Wales',
       'Powys Teaching Health Board'], dtype=object)

In [4]:
beds['GA_C19_percent']  = beds['CO_GABed_C19'] / beds['CO_GABed_Tot']
beds['IV_C19_percent'] = beds['CO_IVBed_C19'] / beds['CO_IVBed_Tot']
beds_percent = beds[['Date', 'health_board', 'GA_C19_percent', 'IV_C19_percent']].copy()
beds_percent.columns = beds_percent.columns.str.lower()
beds_percent.tail()

,date,health_board,ga_c19_percent,iv_c19_percent
2007,2020-11-12,Cwm Taf University Health Board,0.338806,0.488372
2008,2020-11-12,Aneurin Bevan University Health Board,0.231228,0.200000
2009,2020-11-12,Powys Teaching Health Board,0.037975,NaN
2010,2020-11-12,Velindre University NHS Trust,0.000000,NaN
2011,2020-11-12,Wales,0.163741,0.189711


In [5]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_hb_date.csv')
lockdowns['date'] = pd.to_datetime(lockdowns['date'], format = '%d/%m/%Y')
lockdowns.tail()

,date,health_board,lockdown_status
1998,2020-11-07,Wales,No status
1999,2020-11-08,Wales,No status
2000,2020-11-09,Wales,No status
2001,2020-11-10,Wales,No status
2002,2020-11-11,Wales,No status


In [6]:
beds_lockdowns = beds_percent.merge(lockdowns, how = 'inner', on = ['date', 'health_board'])
beds_lockdowns.head()

,date,health_board,ga_c19_percent,iv_c19_percent,lockdown_status
0,2020-04-01,Betsi Cadwaladr University Health Board,0.074519,0.318182,Full national lockdown
1,2020-04-01,Hywel Dda University Health Board,0.082424,0.108696,Full national lockdown
2,2020-04-01,Abertawe Bro Morgannwg University Health Board,0.096339,0.252747,Full national lockdown
3,2020-04-01,Cardiff and Vale University Health Board,0.116950,0.301587,Full national lockdown
4,2020-04-01,Cwm Taf University Health Board,0.116842,0.280702,Full national lockdown


In [7]:
all_wales = beds_lockdowns[beds_lockdowns['health_board']=='Wales']

fig = px.line(all_wales, x = 'date', y = ['ga_c19_percent', 'iv_c19_percent'])
fig.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [8]:
# find the first day of local lockdowns for all healthboards.

def lockdown_countdown(df: pd.DataFrame, hb: str) -> pd.DataFrame:
    temp_df = df[df['health_board'] == hb].sort_values('date')
    first_day = temp_df[temp_df['lockdown_status'] != 'No local lockdowns'].iloc[0,0]
    temp_df = temp_df[temp_df['date']>= first_day - pd.Timedelta('28 days')]
    temp_df = temp_df[temp_df['lockdown_status'] != 'Firebreak lockdown'] 
    temp_df['daycount_from_local_lockdown'] = list(range(-28, len(temp_df)-28))
    return temp_df

countdown_df = pd.concat([lockdown_countdown(beds_lockdowns, hb) for hb in beds_lockdowns.health_board.unique() if hb not in ['Powys Teaching Health Board', 'Velindre University NHS Trust', 'Wales']])
countdown_df.tail()

,date,health_board,ga_c19_percent,iv_c19_percent,lockdown_status,daycount_from_local_lockdown
1810,2020-10-21,Aneurin Bevan University Health Board,0.161251,0.235294,Local lockdowns,175
1819,2020-10-22,Aneurin Bevan University Health Board,0.175353,0.228571,Local lockdowns,176
1981,2020-11-09,Aneurin Bevan University Health Board,0.185340,0.342857,Post-firebreak,177
1990,2020-11-10,Aneurin Bevan University Health Board,0.206794,0.342857,Post-firebreak,178
1999,2020-11-11,Aneurin Bevan University Health Board,0.237187,0.285714,Post-firebreak,179


In [11]:
#all_wales = beds_lockdowns[beds_lockdowns['health_board']=='Wales']

fig = px.line(countdown_df[countdown_df['daycount_from_local_lockdown'] < 56], 
              x = 'daycount_from_local_lockdown', y = 'ga_c19_percent', color = 'health_board')
fig.show()

In [13]:
hbs_filterable = countdown_df[countdown_df['daycount_from_local_lockdown'] < 56].drop(columns = ['date', 'iv_c19_percent', 'lockdown_status'])
hbs_filterable = hbs_filterable[['daycount_from_local_lockdown', 'health_board', 'ga_c19_percent']]
hbs_filterable.head()

,daycount_from_local_lockdown,health_board,ga_c19_percent
0,-28,Betsi Cadwaladr University Health Board,0.074519
8,-27,Betsi Cadwaladr University Health Board,0.054400
16,-26,Betsi Cadwaladr University Health Board,0.066506
24,-25,Betsi Cadwaladr University Health Board,0.071486
32,-24,Betsi Cadwaladr University Health Board,0.084775


In [27]:
hbs_columns = hbs_filterable.pivot(index = 'daycount_from_local_lockdown', columns = 'health_board', values = 'ga_c19_percent') 
hbs_columns.head()

health_board,Abertawe Bro Morgannwg University Health Board,Aneurin Bevan University Health Board,Betsi Cadwaladr University Health Board,Cardiff and Vale University Health Board,Cwm Taf University Health Board,Hywel Dda University Health Board
daycount_from_local_lockdown,,,,,,
-28,0.096339,0.216532,0.074519,0.116950,0.116842,0.082424
-27,0.116552,0.183527,0.054400,0.118959,0.126294,0.084135
-26,0.124758,0.192685,0.066506,0.118431,0.129843,0.093333
-25,0.130159,0.187500,0.071486,0.125833,0.130621,0.076261
-24,0.193267,0.204787,0.084775,0.119911,0.143162,0.090347


In [32]:
import plotly.graph_objects as go
fig = go.Figure()

for column in hbs_columns.columns.to_list():
    fig.add_trace(
        go.Scatter(
            x = hbs_columns.index,
            y = hbs_columns[column],
            name = column
        )
    )
    
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'Abertawe Bro Morgannwg',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Abertawe Bro Morgannwg',
                           'showlegend':True}]),
             dict(label = 'Aneurin Bevan',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False]},
                          {'title': 'Aneurin Bevan',
                           'showlegend':True}]),
             dict(label = 'Betsi Cadwaladr',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False]},
                          {'title': 'Betsi Cadwaladr',
                           'showlegend':True}]),
             dict(label = 'Cardiff and Vale',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False]},
                          {'title': 'Cardiff and Vale',
                           'showlegend':True}]),
             dict(label = 'Cwm Taf',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False]},
                          {'title': 'Cwm Taf',
                           'showlegend':True}]),
             dict(label = 'Hywel Dda',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True]},
                          {'title': 'Hywel Dda',
                           'showlegend':True}]),
            ])
        )
    ])
fig.update_layout(title = 'percentage of general/acute beds used by COVID-19 patients')

fig.write_html('visualisations/lockdown_countdown_gabeds.html')
fig.show()